In [ ]:
import datetime
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
import glob
import shutil
import os
from pathlib import Path

In [ ]:
def navigate_site(username, password, original_path):
    os.chdir(original_path)
    folder = os.path.abspath(os.curdir)
    path = os.path.abspath(folder + '/chromedriver')
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.add_argument('--log-level=3')
    # options.add_experimental_option("detach", True)
    driver = webdriver.Chrome(executable_path=path, options=options)
    url = 'https://partners.wayfair.com/v/login/index'
    driver.get(url)
    try:
        login = driver.find_element_by_id('js-username')
        login.send_keys(username)
        password_element = driver.find_element_by_id('password_field')
        password_element.send_keys(password)
        signin = driver.find_element_by_id('login')
        signin.click()
    except:
        pass
    try:
        login = driver.find_element_by_id('username')
        login.send_keys(username)
        password_element = driver.find_element_by_id('password')
        password_element.send_keys(password)
        signin = driver.find_element_by_id('kc-login')
        signin.click()
    except:
        pass
    time.sleep(3)
    inventory_url = 'https://partners.wayfair.com/v/wfs/products/product/index'
    driver.get(inventory_url)
    time.sleep(3)
    switch_button = driver.find_element_by_css_selector('[data-test-id="changeSupplierButton"]')
    switch_button.click()
    time.sleep(3)
    US_button = driver.find_element_by_xpath('//div[starts-with(text(), "MaloufFineLinens")]')
    US_button.click()
    download_files(driver, original_path, '/US.csv')
    os.chdir(original_path)
    os.chdir('../../../../../../Project Files/Function_Projects/Wayfair_Uploads/Data/Input')
    EU_button = driver.find_element_by_xpath('//div[contains(text(), "EUMaloufFineLinens")]')
    EU_button.click()
    download_files(driver, original_path, '/EU.csv')
    CAN_button = driver.find_element_by_xpath('//div[contains(text(), "CAN_MaloufFineLinens")]')
    CAN_button.click()
    download_files(driver, original_path, '/CAN.csv')

    return driver

In [ ]:
def download_files(driver, original_path, name):
    time.sleep(40)
    attempts = 0
    while attempts < 4:
        try:
            export_button = driver.find_element_by_xpath('//button[contains(text(), "Export CSV")]')
            attempts += 4
        except:
            time.sleep(2)
            attempts += 1
            print('Could not find export button')
    export_button.click()
    time.sleep(15)
    attempts = 0
    while attempts < 4:
        try:
            download_button = driver.find_element_by_xpath('//button[contains(text(), "Download")]')
            attempts += 4
        except:
            time.sleep(15)
            attempts += 1
            print('Could not find download button')
            export_button = driver.find_element_by_xpath('//button[contains(text(), "Export CSV")]')
            export_button.click()
    download_button.click()
    time.sleep(50)
    attempts = 0
    while attempts < 4:
        try:
            dismiss_button = driver.find_element_by_css_selector('[class="DismissButton__IconWrap-sc-1i1c6l4-0 eCUajQ"]')
            dismiss_button.click()

            button = driver.find_element_by_xpath(
            '/html/body/div[2]/div/div/div/div/div[2]/div[1]/div/div/div/div[2]/div[1]/div/button/div[1]')
            time.sleep(15)
            driver.get('https://partners.wayfair.com/v/supplier/download_center/management/app')
            time.sleep(50)
            download_button = driver.find_element_by_xpath(
            "/html/body/div[1]/div/div/div[2]/div[2]/div/div/div/div[3]/div[1]/div/div/div[2]/table/tbody/tr[1]/td[6]/span"
            )
            download_button.click()
            driver.get('https://partners.wayfair.com/v/wfs/products/product/index')
            time.sleep(20)
            print(1)
            attempts += 5
        except:
            time.sleep(20)
            attempts += 1
            pass
    print(2)
    move_files(original_path, name)
    print(3)
    attempts = 0
    while attempts < 4:
        try:
            switch_button = driver.find_element_by_css_selector('[data-test-id="changeSupplierButton"]')
            attempts += 4
        except:
            attempts += 1
            print('Could not find switch button')
    switch_button.click()
    time.sleep(3)
    print(4)
    return

In [ ]:
def move_files(original_path, name):
    folder = str(Path.home() / 'Downloads')
    files = glob.glob(folder + '//*csv')
    top_file = max(files, key = os.path.getctime)
    os.rename(top_file, folder + name)
    file = folder + name
    os.chdir(original_path)
    os.chdir('../../../../../../Project Files/Function_Projects/Wayfair_Uploads/Data/Input')
    folder = os.path.abspath(os.curdir)
    destination = folder + '/'
    shutil.move(file, destination)

In [ ]:
def main(username, password, original_path):
    navigate_site(username, password, original_path)
    print('Successfully downloaded Castlegate Inventory files')

In [ ]:
if __name__ == '__main__':
    main()